In [1]:
#import libraries
import requests
import re
from bs4 import BeautifulSoup
from requests import get
url = "https://hackr.io/tutorials/learn-drawing"
response = get(url)
#We’ll now create a BeautifulSoup object, or a parse tree. This object takes as its arguments the page.text document from Requests (the content of the server’s response) and then parses it from Python’s built-in html.parser.
soup = BeautifulSoup(response.text, 'html.parser')

# Pull all text from the tuturial divison that contains course listing
courses = soup.find_all('div', class_ ="tut-list tut-list-new tut-row")
print(type(courses))
print(len(courses))

<class 'bs4.element.ResultSet'>
6


Now we’ll select each container, and extract, by turn, each item of interest:

- Course title
- Votes
- Author
- Links (Course, Author, Hackr)
- Course Features: (Free / Paid) (video / book ) (Beginner / Intermediate / Advanced)


In [2]:
# Lists to store the scraped data in
CourseTitles = []
Votes = []
CourseLink = []
AuthorName = []
AuthorLink = []
HackrLink = []
CourseCost = []
CourseType = []
StudentType = []


    
for course in courses:
    CourseTitle = course.div.find('span', class_ = 'tutorial-title-txt').text.strip()
    CourseTitles.append(CourseTitle)


    NoOfVote  = course.div.find('span', class_ = 'count').text.strip()
    Votes.append(NoOfVote)

    Link1 = course.find("div", class_="title-links no-margin").find('a')["href"]
    CourseLink.append(Link1)
    
    Name = course.find("a", class_="author no-marginright paddingleft-xs")
    if Name is None:
        AuthorName.append(None)
    else:
        Names=Name.text.strip()
        AuthorName.append(Names)
    
    auth  = course.find("div", class_="action author").find('a')
    if auth is None:
        AuthorLink.append(None)
    else:
        Link2 = auth["href"]
        AuthorLink.append(Link2)
    
    hackr = course.find("a", attrs={'href': re.compile("https://hackr.io/tutorial/")}) 
    if hackr is None:
        HackrLink.append(None)
    else:
        Link3=hackr.get("href")
        HackrLink.append(Link3)
        
    features  = course.find_all('div', class_ = "tut-label-group marginright-sm")
    for future in features:
        x=future.text.split()
    feat1 = x[0]
    if len(x) > 2:
        feat2 = x[1]
        feat3 = x[2]
    elif x[1] == ("Beginner" or "Advanced"):
        feat2 =None
        feat3= x[1]
    else:
        feat2 = x[1]
        feat3 = None
    CourseCost.append(feat1)
    CourseType.append(feat2)
    StudentType.append(feat3)
        

print (f"Courses : {CourseTitles}\n")
print (f"Votes : {Votes}\n")
print (f"CourseLink : {CourseLink}\n")
print (f"Author : {AuthorName}\n")
print (f"AuthorLink : {AuthorLink}\n")
print (f"HackrLink : {HackrLink}\n")
print (f"CourseLink : {CourseLink}\n")
print (f"CourseCost : {CourseCost}\n")
print (f"CourseType : {CourseType}\n")
print (f"StudentType : {StudentType}\n")


Courses : ['Drawing Lesson For Beginners', 'Drawing Lessons for Beginners', 'Manga Workshop Series', 'How to Draw', 'The Ultimate Drawing Course - Beginner to Advanced', 'Character Art School: Complete Character Drawing']

Votes : ['+1', '2', '2', '1', '+1', '+1']

CourseLink : ['https://click.linksynergy.com/deeplink?id=jU79Zysihs4&mid=39197&murl=https://www.udemy.com/drawing-lesson-for-beginners', 'https://www.youtube.com/playlist?list=PLhnHFsOn0cgdjNrlX4qeytqAUIhRqKbAI', 'https://www.youtube.com/playlist?list=PLQbplbxfLmL2ar2TN8ZcNzx7pq1Z4tt6t', 'https://www.youtube.com/playlist?list=PL836C92C18EEB374E', 'https://click.linksynergy.com/deeplink?id=jU79Zysihs4&mid=39197&murl=https://www.udemy.com/the-ultimate-drawing-course-beginner-to-advanced', 'https://click.linksynergy.com/deeplink?id=jU79Zysihs4&mid=39197&murl=https://www.udemy.com/character-art-school-complete-character-drawing']

Author : ['Eric', 'Pawan', None, 'Jack', 'Jonathan', 'Rajat']

AuthorLink : ['https://hackr.io/eric

In [3]:
import pandas as pd
DrawingCourses = pd.DataFrame({'Course': CourseTitles,
'Votes': Votes,
"Course Link":CourseLink,
"Author ":AuthorName,
"Author Link":AuthorLink,
"Hackr Link":HackrLink,
"Course Cost":CourseCost,
"Course Type":CourseType,
"Course Cost":CourseCost,
"Student Type":StudentType
})
print(DrawingCourses.info())
DrawingCourses

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Course        6 non-null      object
 1   Votes         6 non-null      object
 2   Course Link   6 non-null      object
 3   Author        5 non-null      object
 4   Author Link   5 non-null      object
 5   Hackr Link    5 non-null      object
 6   Course Cost   6 non-null      object
 7   Course Type   6 non-null      object
 8   Student Type  5 non-null      object
dtypes: object(9)
memory usage: 560.0+ bytes
None


,Course,Votes,Course Link,Author,Author Link,Hackr Link,Course Cost,Course Type,Student Type
0,Drawing Lesson For Beginners,+1,https://click.linksynergy.com/deeplink?id=jU79...,Eric,https://hackr.io/eric-bossik,https://hackr.io/tutorial/drawing-lesson-for-b...,Paid,Video,Beginner
1,Drawing Lessons for Beginners,2,https://www.youtube.com/playlist?list=PLhnHFsO...,Pawan,https://hackr.io/pawan-kumar-2,https://hackr.io/tutorial/drawing-lessons-for-...,Free,Video,Beginner
2,Manga Workshop Series,2,https://www.youtube.com/playlist?list=PLQbplbx...,None,None,None,Free,Video,Beginner
3,How to Draw,1,https://www.youtube.com/playlist?list=PL836C92...,Jack,https://hackr.io/jack-avery-1,https://hackr.io/tutorial/how-to-draw,Free,Video,Beginner
4,The Ultimate Drawing Course - Beginner to Adva...,+1,https://click.linksynergy.com/deeplink?id=jU79...,Jonathan,https://hackr.io/jonathan-cumberbatch,https://hackr.io/tutorial/the-ultimate-drawing...,Paid,Video,None
5,Character Art School: Complete Character Drawing,+1,https://click.linksynergy.com/deeplink?id=jU79...,Rajat,https://hackr.io/rajat-patel-3,https://hackr.io/tutorial/character-art-school...,Advanced,Paid,Video


In [4]:
DrawingCourses.to_csv('DrawingCourses.csv')